In [1]:
import os, sys, re, io, math
import matplotlib, shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib.patches import Patch
from matplotlib.widgets import Slider
from collections import *
from math import isnan, nan
from glob import glob
from ipywidgets import *
from datetime import datetime, timedelta
from termcolor import colored
import ipywidgets as widgets
from IPython.display import *
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.width = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100000
pd.options.display.float_format = '{0:f}'.format

# Disable scrolling for all output widgets
style = """
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
        .jupyter-widgets-output-area  {
            height: auto !important;
        }
    </style>
    """
display(HTML(style))

# disable autosave so that multiple user browser can attach the same notebook
%autosave 0

from core import *

# enable notebook mode so that we can drag, resize and save plots, but plot will be smaller
# %matplotlib notebook

Autosave disabled


<IPython.core.display.Javascript object>

In [2]:
# main file path
PATH="./3.decrypted/izedAa85XXrDS85XlwrOsIDU"

In [4]:
def on_button_clicked(b):
    for userpath in os.listdir(PATH):
        if os.path.isdir(PATH+'/'+userpath):
            print('Checking %s ...' % userpath)
            df_taps = load_df(PATH+'/'+userpath+'/tapsLog.csv.gz', '', -1)
            df_acctap = load_df(PATH+'/'+userpath+'/accessibilityLog.csv.gz', '', -1)
            df_socmsg = load_df(PATH+'/'+userpath+'/sociabilityMsgLog.csv.gz', '', -1)
            df_socall = load_df(PATH+'/'+userpath+'/sociabilityCallLog.csv.gz', '', -1)
            if len(df_socmsg.index) == len(df_socall.index) == 0:
                df_whatsapp_taps = Try(lambda: df_taps[df_taps.in_app_name.str.contains('whatsapp')], pd.DataFrame())
                if len(df_whatsapp_taps.index)>0:
                    print('n_socmsg=%d, n_socall=%s, but n_whatsapp_taps=%d'%(len(df_socmsg.index), len(df_socall.index), len(df_whatsapp_taps.index)))
                    print(df_whatsapp_taps)
            if len(df_taps.index)>0 and len(df_acctap.index)==0:
                print('n_taps=%d but n_acctap=%d'%(len(df_taps.index), len(df_acctap.index)))
    print('All finished!')
button = widgets.Button(description='Check Consistency', style={'font_weight': 'bold'})
button.on_click(on_button_clicked)
display(button)

Button(description='Check Consistency', style=ButtonStyle(font_weight='bold'))

Checking moht.dsth.009@moht.com.sg_56f4de7d50fa ...
Checking moht.dsth.022@moht.com.sg_8c89fea59c22 ...
Checking moht.dsth.035@moht.com.sg_4893c9610705 ...
Checking moht.dsth.003@moht.com.sg_cdb01556d05e ...
Checking moht.dsth.006@moht.com.sg_a23b045b6402 ...
Checking moht.dsth.026@moht.com.sg_e5f8902bafb5 ...
Checking moht.dsth.031@moht.com.sg_24b9427bb09c ...
Checking moht.dsth.017@moht.com.sg_e551519f7675 ...
Checking moht.dsth.149@moht.com.sg_715aba340a7f ...
Checking moht.dsth.039@moht.com.sg_c18f790c477e ...
Checking moht.dsth.004@moht.com.sg_77c22e42696d ...
Checking moht.dsth.001@moht.com.sg_a8077a5dd3e8 ...
Checking moht.dsth.024@moht.com.sg_a25126125e7e ...
Checking moht.dsth.021@moht.com.sg_68468a4a3007 ...
Checking moht.dsth.020@moht.com.sg_5842db147d01 ...
Checking moht.dsth.012@moht.com.sg_69ed90f14d4f ...
Checking moht.dsth.027@moht.com.sg_2cbea652ed82 ...
Checking moht.dsth.008@moht.com.sg_68e8d4623416 ...
Checking moht.dsth.015@moht.com.sg_a4c08a5b2a9f ...
Checking moh